In [ ]:
%run fungsi_save.ipynb

In [ ]:
#pilih data fitur masukan NN1
train_data = pd.read_csv('data_training.csv', sep=',',header=None)
train_data = train_data.values
post = pd.read_csv('data_featureclass_pos.csv', sep=',',header=None)
post = np.int_(post)
total_features = np.size(train_data, axis=1)-2
print(total_features)
output_train_data = train_data[:,total_features] #output aja lym
output_train_data = np.int_(output_train_data)
train_data = train_data[:,:total_features]
output_train_data

In [ ]:
a = [] #index fitur yang gak kepake
for i in range(len(post)):
    if post[i,0] == 0:
        a.append(i)
train_data = np.delete(train_data, a ,axis=1) #data fitur yang udah terseleksi
print("Number of training samples is", len(train_data))
print(np.size(train_data,axis=1))
print(train_data[0])

In [ ]:
#data validasi
validasi_data = pd.read_csv('data_testing.csv', sep=',',header=None)
validasi_data = validasi_data.values
output_validasi_data = validasi_data[:,total_features] #output aja untuk output 1
output_validasi_data = np.int_(output_validasi_data)
validasi_data = validasi_data[:,0:total_features] #fitur aja tanpa output dan fiturnya lengkap
validasi_data = np.delete(validasi_data, a ,axis=1) #data fitur yang udah terseleksi

#save_data1(test_data)

In [ ]:
n_inputs = np.size(validasi_data,1)
n_hidden = 20
n_classes = 3
n_inputs

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(output_train_data)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
labels = np.array(onehot_encoded)#output_train_data) 

In [ ]:
train_data = train_data/np.amax(train_data, axis=0) 

In [ ]:
error1 = []
@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
class NeuralNetwork:
        
    
    def __init__(self, 
                 no_of_in_nodes, 
                 no_of_out_nodes, 
                 no_of_hidden_nodes,
                 learning_rate,
                 bias=None
                ):  
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_out_nodes = no_of_out_nodes
        
        self.no_of_hidden_nodes = no_of_hidden_nodes
            
        self.learning_rate = learning_rate 
        self.bias = bias
        self.create_weight_matrices()
    
        
    
    def create_weight_matrices(self):
        """ A method to initialize the weight matrices of the neural 
        network with optional bias nodes"""
        
        bias_node = 1 if self.bias else 0
        
        rad = 1 / np.sqrt(self.no_of_in_nodes + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.weights_in_hidden = X.rvs((self.no_of_hidden_nodes, 
                                       self.no_of_in_nodes + bias_node))
        rad = 1 / np.sqrt(self.no_of_hidden_nodes + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.weights_hidden_out = X.rvs((self.no_of_out_nodes, 
                                        self.no_of_hidden_nodes + bias_node))
        
        
        
    def train(self, input_vector, target_vector):
        # input_vector and target_vector can be tuple, list or ndarray
        global error1
        bias_node = 1 if self.bias else 0
        if self.bias:
            # adding bias node to the end of the inpuy_vector
            input_vector = np.concatenate( (input_vector, [self.bias]) )
                                    
            
        input_vector = np.array(input_vector, ndmin=2).T
        target_vector = np.array(target_vector, ndmin=2).T
        
        output_vector1 = np.dot(self.weights_in_hidden, input_vector)
        output_vector_hidden = activation_function(output_vector1)
        
        if self.bias:
            output_vector_hidden = np.concatenate( (output_vector_hidden, [[self.bias]]) )
        
        
        output_vector2 = np.dot(self.weights_hidden_out, output_vector_hidden)
        output_vector_network = activation_function(output_vector2)
        
        output_errors = target_vector - output_vector_network
        error1.append(output_errors)
        self.output_errors = output_errors
        # update the weights:
        tmp = output_errors * output_vector_network * (1.0 - output_vector_network)     
        tmp = self.learning_rate  * np.dot(tmp, output_vector_hidden.T)
        self.weights_hidden_out += tmp
        # calculate hidden errors:
        hidden_errors = np.dot(self.weights_hidden_out.T, output_errors)
        # update the weights:
        tmp = hidden_errors * output_vector_hidden * (1.0 - output_vector_hidden)
        if self.bias:
            x = np.dot(tmp, input_vector.T)[:-1,:]     # ???? last element cut off, ???
        else:
            x = np.dot(tmp, input_vector.T)
        self.weights_in_hidden += self.learning_rate * x
    def error(self):
        return self.output_errors
    def run(self, input_vector):
        # input_vector can be tuple, list or ndarray
        
        if self.bias:
            # adding bias node to the end of the inpuy_vector
            input_vector = np.concatenate( (input_vector, [1]) )
        input_vector = np.array(input_vector, ndmin=2).T
        output_vector = np.dot(self.weights_in_hidden, input_vector)
        output_vector = activation_function(output_vector)
        
        if self.bias:
            output_vector = np.concatenate( (output_vector, [[1]]) )
            
        output_vector = np.dot(self.weights_hidden_out, output_vector)
        output_vector = activation_function(output_vector)
        output_vector2 = np.argmax(output_vector, axis=0)
        W1 = self.weights_in_hidden
        W2 = self.weights_hidden_out
        save(W1,W2)
        return output_vector2

In [ ]:
simple_network = NeuralNetwork(no_of_in_nodes=n_inputs, 
                               no_of_out_nodes=3, 
                               no_of_hidden_nodes=n_hidden,
                               learning_rate=0.1,
                               bias=None)
output_predict_train = []
error = []
for z in range(300):
    for i in range(len(train_data)):
        simple_network.train(train_data[i], labels[i])
    print(simple_network.error())
    error.append(error)

In [ ]:
for i in range(len(train_data)):
    print("data ke-",i)
    print(output_train_data[i])
    print(simple_network.run(train_data[i]))
    output_predict_train.append(simple_network.run(train_data[i]))

In [ ]:
output_predict_train = np.array(output_predict_train)

In [ ]:
validasi_data = validasi_data/np.amax(validasi_data, axis=0) 

In [ ]:
output_predict_validasi = []
for i in range(len(validasi_data)):
    print(i)
    print(output_validasi_data[i])
    print(simple_network.run(validasi_data[i]))
    output_predict_validasi.append(simple_network.run(validasi_data[i]))

In [ ]:
output_predict_validasi = np.array(output_predict_validasi)

In [ ]:
cm_train = confusion_matrix(output_train_data, output_predict_train)
cm_validasi = confusion_matrix(output_validasi_data, output_predict_validasi)

In [ ]:
print("Presisi data training limfoblas : ", precision(1,cm_train))
print("Recall data training limfoblas (sensitivitas) : ", recall(1,cm_train))
print("Presisi rerata data training : ", precision_macro_average(cm_train))
print("Presisi rerata data training : ", recall_macro_average(cm_train))
print("Akurasi data training : ", accuracy(cm_train))
print("F1 Score data training : ", f1_score(precision_macro_average(cm_train),recall_macro_average(cm_train)))

In [ ]:
print("Presisi data validasi limfoblas : ", precision(1,cm_validasi))
print("Recall data validasi limfoblas", recall(1,cm_validasi))
print("Presisi rerata data validasi : ", precision_macro_average(cm_validasi))
print("Presisi rerata data validasi : ", recall_macro_average(cm_validasi))
print("Akurasi data validasi : ", accuracy(cm_validasi))
print("F1 Score data validasi : ", f1_score(precision_macro_average(cm_validasi),recall_macro_average(cm_validasi)))

In [ ]:
error1 = np.array(error1)

In [ ]:
epoch =np.array([i for i in range(300)])

In [ ]:
cost = 0
cost_er = []
for i in range(300):
    cost = 0
    for z in range(300):
        cost = cost+abs(error1[z+(len(train_data)*i)])
    cost_er.append(cost)

In [ ]:
cost_er = np.array(cost_er)
cost_er.shape

In [ ]:
cost_error = []
for i in range(300):
    cost_error.append((np.mean(cost_er[i]/300,axis=0)))

In [ ]:
cost_error = np.array(cost_error)

In [ ]:
plt.plot(epoch,cost_error)
plt.show()

In [ ]:
cm_t.figure.savefig('fix2(balanced)/bp/train4.png')
cm_v.figure.savefig('fix2(balanced)/bp/validasi4.png')
save_pos_step1('fix2(balanced)/bp/perfo_bp.csv',(accuracy(cm_train),accuracy(cm_validasi),cost_error))